In [7]:
# Imports
import numpy as np
import csv

files = [
    "../human_score/quiz_sp_sr_mc.tsv",
    "../human_score/quiz_sp_cr_mc.tsv",
    "../human_score/quiz_sp_base_mc.tsv",
]

results = []

for file in files:
    with open(file) as tsvfile:
        res = []
        reader = csv.reader(tsvfile, delimiter='\t')
        next(reader)
        for row in reader:
            res.append(row[1:])
        results.append(res)

In [9]:
import json

# Read in answers
with open("../data/SP/SP_test_clean.json", "r", encoding='utf-8') as f:
    data = json.load(f)
sr = []
cr = []
base = []
for d in data:
    if "SR" in d["id"]:
        sr.append(d)
    elif "CR" in d["id"]:
        cr.append(d)
    else:
        base.append(d)

# Create dictionary for results
sr_results = []
for i in range(len(results[0][0])): # length of questions
    ans = []
    for j in range(len(results[0])): # length of respondents
        if results[0][j][i] not in sr[i]["choice_list"]:
            ans.append(4)
        else: 
            for x in range(4):
                if results[0][j][i] == sr[i]["choice_list"][x]:
                    ans.append(x)
    sr_results.append({
        "id": sr[i]["id"],
        "question": sr[i]["question"],
        "label": sr[i]["label"],
        "human_score": ans
    })
cr_results = []
for i in range(len(results[1][0])): # length of questions
    ans = []
    for j in range(len(results[1])): # length of respondents
        if results[1][j][i] not in cr[i]["choice_list"]:
            ans.append(4)
        else: 
            for x in range(4):
                if results[1][j][i] == cr[i]["choice_list"][x]:
                    ans.append(x)
    cr_results.append({
        "id": cr[i]["id"],
        "question": cr[i]["question"],
        "label": cr[i]["label"],
        "human_score": ans
    })
base_results = []
for i in range(len(results[2][0])): # length of questions
    ans = []
    for j in range(len(results[2])): # length of respondents
        if results[2][j][i] not in base[i]["choice_list"]:
            ans.append(4)
        else: 
            for x in range(4):
                if results[2][j][i] == base[i]["choice_list"][x]:
                    ans.append(x)
    base_results.append({
        "id": base[i]["id"],
        "question": base[i]["question"],
        "label": base[i]["label"],
        "human_score": ans
    })
outfiles_all = [
    "../human_score/scores_sp_sr_mc.json",
    "../human_score/scores_sp_cr_mc.json",
    "../human_score/scores_sp_base_mc.json",
]
results_items = [sr_results, cr_results, base_results]
for file, item in zip(outfiles_all, results_items):
    with open(file, "w", encoding='utf-8') as f:
        json.dump(item, f, ensure_ascii=False, indent=4)

In [13]:
# Scoring function
def getScores(data, index=None):
    """
    scores: list of scores for each participant
    avg_score: average score for all participants
    group_score: only counts if all participants get the correct answer
    agg_score: counts if 2/3 got the correct answer
    max_score: counts if a single participant got the correct answer
    unsure (4) counts as none of the above (3)
    """
    if index is not None:
        p_num = len(index)
    else:
        p_num = len(data[0]["human_score"])
    scores = np.array([0]*p_num)
    group_score = 0
    agg_score = 0
    max_score = 0
    for item in data:
        group = 0
        if index is None:
            if item["label"] in item["human_score"]:
                max_score += 1
            for i in range(p_num):
                if item["human_score"][i] == item["label"]:
                    scores[i] += 1
                    group += 1
                elif item["human_score"][i] == 4 and item["label"] == 3:
                    scores[i] += 1
                    group += 1
            if group == p_num:
                group_score += 1
        else:
            index_scores = [item["human_score"][j] for j in index]
            if item["label"] in index_scores:
                max_score += 1
            for j, i in enumerate(index):
                if item["human_score"][i] == item["label"]:
                    scores[j] += 1
                    group += 1
                elif item["human_score"][i] == 4 and item["label"] == 3:
                    scores[j] += 1
                    group += 1
            if group == p_num:
                group_score += 1
        if group >= 2:
            agg_score += 1
    return ["%5.1f" % (score/0.4) for score in scores], "%5.1f" % (np.average(scores)/0.4), "%5.1f" % (group_score/0.4), "%5.1f" % (agg_score/0.4), "%5.1f" % (max_score/0.4)

base_scores, base_avg_score, base_group_score, base_agg_score, base_max_score = getScores(base_results)
sr_scores, sr_avg_score, sr_group_score, sr_agg_score, sr_max_score = getScores(sr_results)
cr_scores, cr_avg_score, cr_group_score, cr_agg_score, cr_max_score = getScores(cr_results)

print("All Data:")
print("Base scores:", base_scores, base_avg_score, base_group_score, base_max_score)
print("SR scores:", sr_scores, sr_avg_score, sr_group_score, sr_max_score)
print("CR scores:", cr_scores, cr_avg_score, cr_group_score, cr_max_score)
print("Chosen 3 Participants:")
print("Base chosen:", getScores(base_results, [2, 3, 4]))
print("SR chosen:", getScores(sr_results, [0, 1, 2]))
print("CR chosen:", getScores(cr_results, [1, 2, 3]))
print("Closest Mean 3: ")
print("Base mean:", getScores(base_results, [1, 3, 4]))
print("SR mean:", getScores(sr_results, [0, 2, 3]))
print("CR mean:", getScores(cr_results, [1, 2, 3]))

All Data:
Base scores: [' 80.0', ' 82.5', ' 97.5', ' 85.0', ' 85.0']  86.0  57.5 100.0
SR scores: [' 77.5', ' 65.0', ' 87.5', ' 92.5']  80.6  55.0  97.5
CR scores: [' 77.5', ' 60.0', ' 57.5', ' 62.5']  64.4  30.0  82.5
Chosen 3 Participants:
Base chosen: ([' 97.5', ' 85.0', ' 85.0'], ' 89.2', ' 75.0', ' 92.5', '100.0')
SR chosen: ([' 77.5', ' 65.0', ' 87.5'], ' 76.7', ' 55.0', ' 77.5', ' 95.0')
CR chosen: ([' 60.0', ' 57.5', ' 62.5'], ' 60.0', ' 30.0', ' 65.0', ' 80.0')
Closest Mean 3: 
Base mean: ([' 82.5', ' 85.0', ' 85.0'], ' 84.2', ' 65.0', ' 87.5', '100.0')
SR mean: ([' 77.5', ' 87.5', ' 92.5'], ' 85.8', ' 70.0', ' 90.0', ' 97.5')
CR mean: ([' 60.0', ' 57.5', ' 62.5'], ' 60.0', ' 30.0', ' 65.0', ' 80.0')


In [2]:
import json

with open("../data/SP/SP_train_clean.json", "r", encoding='utf-8') as f:
    train_data = json.load(f)
print(len(train_data))

507


In [5]:
import json

# Read in answers
with open("../data/SP/SP_test_clean.json", "r", encoding='utf-8') as f:
    data = json.load(f)
sr = []
cr = []
base = []
for d in data:
    if "SR" in d["id"]:
        sr.append(d)
    elif "CR" in d["id"]:
        cr.append(d)
    else:
        base.append(d)
# Create dictionary for results
sr_results = []
for i in range(len(results[0][0])): # length of questions
    ans = []
    for j in [0, 2, 3]: # length of respondents
        if results[0][j][i] not in sr[i]["choice_list"]:
            ans.append(4)
        else: 
            for x in range(4):
                if results[0][j][i] == sr[i]["choice_list"][x]:
                    ans.append(x)
    sr_results.append({
        "id": sr[i]["id"],
        "question": sr[i]["question"],
        "label": sr[i]["label"],
        "human_score": ans
    })
cr_results = []
for i in range(len(results[1][0])): # length of questions
    ans = []
    for j in [1, 2, 3]: # length of respondents
        if results[1][j][i] not in cr[i]["choice_list"]:
            ans.append(4)
        else: 
            for x in range(4):
                if results[1][j][i] == cr[i]["choice_list"][x]:
                    ans.append(x)
    cr_results.append({
        "id": cr[i]["id"],
        "question": cr[i]["question"],
        "label": cr[i]["label"],
        "human_score": ans
    })
base_results = []
for i in range(len(results[2][0])): # length of questions
    ans = []
    for j in [1, 3, 4]: # length of respondents
        if results[2][j][i] not in base[i]["choice_list"]:
            ans.append(4)
        else: 
            for x in range(4):
                if results[2][j][i] == base[i]["choice_list"][x]:
                    ans.append(x)
    base_results.append({
        "id": base[i]["id"],
        "question": base[i]["question"],
        "label": base[i]["label"],
        "human_score": ans
    })
outfiles_all = [
    "../human_score/scores_sp_sr_mc_final.json",
    "../human_score/scores_sp_cr_mc_final.json",
    "../human_score/scores_sp_base_mc_final.json",
]
results_items = [sr_results, cr_results, base_results]
for file, item in zip(outfiles_all, results_items):
    with open(file, "w", encoding='utf-8') as f:
        json.dump(item, f, ensure_ascii=False, indent=4)

all_results = []
sr_id = [x["id"] for x in sr_results]
cr_id = [x["id"] for x in cr_results]
base_id = [x["id"] for x in base_results]
for item in data:
    if item["id"] in sr_id:
        all_results.append(sr_results[sr_id.index(item["id"])])
    elif item["id"] in cr_id:
        all_results.append(cr_results[cr_id.index(item["id"])])
    else:
        all_results.append(base_results[base_id.index(item["id"])])
with open ("../human_score/scores_sp_combined_final.json", "w", encoding='utf-8') as f:
    json.dump(all_results, f, ensure_ascii=False, indent=4)

In [6]:
from scipy import stats as st

answers = []
for item in all_results:
    if item["human_score"][0] != item["human_score"][1] and item["human_score"][1] != item["human_score"][2] and item["human_score"][2] != item["human_score"][0]:
        answers.append('')
        print(str(item["id"]) + ": " + str(item["human_score"]) + " " + str(item["label"]))
    else:
        mode = st.mode(item["human_score"])[0]
        if mode == 4:
            mode = 3
        answers.append(mode)
# with open ("../human_score/answer_sen.txt", "w") as f:
#     for item in answers:
#         f.write(str(item) + "\n")

SP-120_CR: [1, 3, 4] 0
SP-156_SR: [0, 3, 4] 2
SP-47_SR: [3, 4, 1] 1
SP-182_CR: [1, 3, 4] 3
SP-189_CR: [2, 3, 0] 0
SP-72: [0, 2, 1] 0
SP-187_SR: [1, 4, 3] 3
SP-59_CR: [3, 1, 4] 1
SP-33_CR: [0, 4, 2] 0
SP-158_CR: [0, 4, 3] 3
SP-141: [2, 1, 3] 3
SP-187_CR: [2, 4, 3] 3
